In [0]:
import numpy as np
import pandas as pd

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# PREPROCESSING


## Import e unione
- import di tutti i dataset divisi per hashtag cercato, 
- unione csv in 2 dataset distinti per hashtag

In [0]:
chec1 = pd.read_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/CheccoZalone/checco_03-01.csv")
chec2 = pd.read_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/CheccoZalone/checco_04-01.csv")
chec3 = pd.read_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/CheccoZalone/checco_06-01_mattino.csv")
chec4 = pd.read_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/CheccoZalone/checco_06-01_sera.csv")
chec5 = pd.read_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/CheccoZalone/checco_07-01.csv")
chec6 = pd.read_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/CheccoZalone/checco_09-01.csv")

tolo1 = pd.read_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/ToloTolo/tolotolo_03-01.csv")
tolo2 = pd.read_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/ToloTolo/tolotolo_04-01.csv")
tolo3 = pd.read_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/ToloTolo/tolotolo_05-01.csv")
tolo4 = pd.read_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/ToloTolo/tolotolo_06-01.csv")
tolo5 = pd.read_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/ToloTolo/tolotolo_06-01-sera.csv")
tolo6 = pd.read_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/ToloTolo/tolotolo_07-01.csv")
tolo7 = pd.read_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/ToloTolo/tolotolo_09-01.csv")

tutto_chec = pd.concat([chec1,chec2,chec3,chec4,chec5,chec6])
tutto_tolo = pd.concat([tolo1,tolo2,tolo3,tolo4,tolo5,tolo6,tolo7])

print(tutto_chec.shape)
print(tutto_tolo.shape)

Creazione dataset *tolotolo.csv* e *checcozalone.csv*

In [0]:
tutto_tolo.to_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/tolotolo.csv")
tutto_chec.to_csv("drive/My Drive/SOCIAL MEDIA ANALYTICS/Progetto/tweet_csv/checcozalone.csv")

Creazione dataset *uniti.csv*

In [0]:
uniti = pd.concat([tutto_tolo,tutto_chec])


## Rimozione tweet duplicati

Si prendono i tweet che possiedono tutti i campi uguali che si differenziano solamente per il *retweet_count*, si seleziona il campo con il valore più alto.
Motivo operazione: raccolta avvenuta in batch utilizzando come parametro *since* Tweepy 27/12/19



Rimozione tweet identici

In [0]:
uniti = uniti.drop_duplicates(keep='first')

Rimozione tweet che sono uguali in tutti i campi e hanno un *retweet_count* minore del massimo.

In [0]:
ids = set(uniti["id_tweet"])
out1 = pd.DataFrame()
for i in ids: 
  tmp = uniti.loc[uniti["id_tweet"] == i]
  if(len(tmp)>1):
    out1 = pd.concat([out1,tmp.loc[tmp["retweet_count"] == max(tmp["retweet_count"])]])
  else:
    out1 = pd.concat([out1,tmp])

uniti = out1

In [0]:
uniti.to_csv("uniti.csv")

## Costruzione delle feature

In [0]:
day = []
month = []

for i in range(0,len(uniti)):
  m = uniti['date_tweet'][i].split('-')[1]
  if m == '01':
    month.append('Jan')
  else:
    month.append('Dec')
  d = uniti['date_tweet'][i].split('-')[2]
  day.append(d)

uniti['month'] = month
uniti['day'] = day

for i in range(0,len(uniti)):
  if pd.isnull(uniti['retweet_screen_name'][i]):
    uniti['retweet_screen_name'][i] = uniti['user_screen_name'][i]

tweet = uniti

Si eliminano i tweet del 9 gennaio visto che l'analisi viene effettuata dal 27 dicembre a 8 gennaio

In [0]:
tweet = tweet[tweet.day != '09']

## Creazione dataset utilizati per creare la rete e per la Sentiment Analysis

In [0]:
relazioni = tweet.drop(['id_tweet','time_tweet','date_tweet','retweet_count','type'], axis = 'columns')

In [0]:
relazioni.to_csv("relazioni.csv")
tweet.to_csv("tweet.csv")